In [1]:
# ===============================================
# Toss CTR — DCN (Split, Fast & Safe)
# ===============================================
import os, gc, math, random
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import average_precision_score, log_loss
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader




In [2]:
# 0) Repro & Device
SEED = 42
def set_seed(s=SEED):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False
set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 1) Load
train = pd.read_parquet("train_input_2.parquet")
test  = pd.read_parquet("test_input_2.parquet")


/opt/conda/envs/ml_project/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Device: cpu


In [3]:
# 2) Columns
id_cols = [c for c in ["row_id","id"] if c in train.columns or c in test.columns]
id_cols_train = [c for c in id_cols if c in train.columns]
id_cols_test  = [c for c in id_cols if c in test.columns]
target_col = "clicked"
assert target_col in train.columns, "clicked target missing in train_input_2.parquet"

# 3) Feature matrix (handle stray object dtypes)
X = train.drop(columns=id_cols_train + [target_col]).copy()
y = train[target_col].astype(np.float32).values
X_test = test.drop(columns=id_cols_test).copy()

# Convert any object columns to numeric via shared LabelEncoder
for c in X.columns:
    if X[c].dtype == "object" or X_test[c].dtype == "object":
        le = LabelEncoder()
        both = pd.concat([X[c].astype(str), X_test[c].astype(str)], axis=0)
        le.fit(both)
        X[c] = le.transform(X[c].astype(str))
        X_test[c] = le.transform(X_test[c].astype(str))

# Ensure float32 tensors
X = X.astype(np.float32)
X_test = X_test.astype(np.float32)

X = X.clip(-3, 3).fillna(0)
X_test = X_test.clip(-3, 3).fillna(0)

FEATURES = X.columns.tolist()
print(f"Features: {len(FEATURES)} | Pos ratio: {y.mean():.4f}")



Features: 26 | Pos ratio: 0.0191


In [4]:
# 4) Split (80/20 stratified)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, val_idx = next(sss.split(X, y))
X_tr, X_va = X.iloc[train_idx].values, X.iloc[val_idx].values
y_tr, y_va = y[train_idx], y[val_idx]

# 5) Datasets
class CTRDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
    def __len__(self): return len(self.X)
    def __getitem__(self, i):
        x = torch.from_numpy(self.X[i])
        if self.y is None: return x
        return x, torch.tensor(self.y[i], dtype=torch.float32)

BATCH = 2048
train_ds = CTRDataset(X_tr, y_tr)
val_ds   = CTRDataset(X_va, y_va)
test_ds  = CTRDataset(X_test.values)

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)



In [5]:
# 6) DCN Model (Cross Network + Deep MLP)
class CrossLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.w = nn.Parameter(torch.randn(dim))
        self.b = nn.Parameter(torch.zeros(dim))
    def forward(self, x0, x):
        # x_{l+1} = x0 * (w^T x) + b + x
        # (batch, dim)
        dot = (x @ self.w)  # (batch,)
        dot = torch.clamp(dot, -3, 3)
        return x0 * dot.unsqueeze(1) + self.b + x

class DCN(nn.Module):
    def __init__(self, dim, cross_layers=3, hidden=(512,256,64), p_drop=0.5):
        super().__init__()
        self.cross = nn.ModuleList([CrossLayer(dim) for _ in range(cross_layers)])
        layers = []
        in_dim = dim
        for h in hidden:
            layers += [nn.Linear(in_dim, h), nn.ReLU(inplace=True), nn.Dropout(p_drop)]
            in_dim = h
        self.deep = nn.Sequential(*layers)
        self.out = nn.Linear(in_dim + dim, 1)  # concatenate cross_out & deep_out
    def forward(self, x):
        x0 = x
        xc = x
        for cl in self.cross:
            xc = cl(x0, xc)
        xd = self.deep(x)
        z = torch.cat([xc, xd], dim=1)
        return self.out(z).squeeze(1)  # logits

in_dim = len(FEATURES)
model = DCN(dim=in_dim, cross_layers=3, hidden=(512,256,64), p_drop=0.3).to(device)

# 7) Loss & Optim
pos = y_tr.sum(); neg = len(y_tr) - pos
pos_weight = torch.tensor((neg / max(1.0, pos)), dtype=torch.float32, device=device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # handles imbalance
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-5)



In [6]:
# 8) Train (Early Stopping on val AUPRC)
EPOCHS = 12
PATIENCE = 3
best_ap = -1.0
best_state = None
no_improve = 0

def eval_loop(loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for batch in loader:
            xb, yb = batch
            xb = xb.to(device)
            logits = model(xb)
            prob = torch.sigmoid(logits).float().cpu().numpy()
            # --- 여기 추가 ---
            prob = np.nan_to_num(prob, nan=0.5, posinf=1.0, neginf=0.0)
            # ----------------
            ps.append(prob)
            ys.append(yb.numpy())
    y_true = np.concatenate(ys)
    y_pred = np.concatenate(ps)
    ap = average_precision_score(y_true, y_pred)
    wll = log_loss(y_true, np.clip(y_pred, 1e-6, 1-1e-6))
    return ap, wll

for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        xb, yb = batch
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = criterion(logits, yb)
        if torch.isnan(loss):
         print("[WARN] NaN loss detected — skipping batch")
         continue
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)

    ap, wll = eval_loop(val_loader)
    print(f"[Epoch {epoch}] train_loss={total_loss/len(train_ds):.6f} | val_AP={ap:.6f} | val_WLL={wll:.6f}")

    if ap > best_ap + 1e-5:
        best_ap = ap
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f"Early stopping at epoch {epoch} (best AP={best_ap:.6f})")
            break

# load best
if best_state is not None:
    model.load_state_dict(best_state)



[Epoch 1] train_loss=1.243238 | val_AP=0.061478 | val_WLL=0.610103
[Epoch 2] train_loss=1.221125 | val_AP=0.061988 | val_WLL=0.639057
[Epoch 3] train_loss=1.217878 | val_AP=0.063414 | val_WLL=0.654988
[Epoch 4] train_loss=1.216086 | val_AP=0.063907 | val_WLL=0.641378
[Epoch 5] train_loss=1.214915 | val_AP=0.063307 | val_WLL=0.616376
[Epoch 6] train_loss=1.213401 | val_AP=0.064629 | val_WLL=0.592755
[Epoch 7] train_loss=1.212522 | val_AP=0.064181 | val_WLL=0.620692
[Epoch 8] train_loss=1.211957 | val_AP=0.064743 | val_WLL=0.630372
[Epoch 9] train_loss=1.211121 | val_AP=0.064231 | val_WLL=0.595105
[Epoch 10] train_loss=1.210466 | val_AP=0.064632 | val_WLL=0.638846
[Epoch 11] train_loss=1.209793 | val_AP=0.064929 | val_WLL=0.605283
[Epoch 12] train_loss=1.209587 | val_AP=0.065267 | val_WLL=0.636680


In [12]:
# ===============================================
# 9️⃣ Predict Test & Save Validation for Blending
# ===============================================
# ===============================================
# 9️⃣ Predict Test & Save Validation for Blending
# ===============================================
model.eval()

# --- Predict test ---
all_probs = []
with torch.no_grad():
    for xb in test_loader:
        xb = xb.to(device)
        prob = torch.sigmoid(model(xb)).float().cpu().numpy()
        all_probs.append(prob)
test_pred = np.concatenate(all_probs)
test_pred = np.clip(test_pred, 1e-4, 1-1e-4)

# --- Evaluate on validation ---
ap, wll = eval_loop(val_loader)
print(f"[Final] val_AP={ap:.6f} | val_WLL={wll:.6f}")

# --- Save validation info for blending ---
np.save("dcn_val_index.npy", val_idx)

val_probs = []
with torch.no_grad():
    for xb, _ in val_loader:
        xb = xb.to(device)
        val_probs.append(torch.sigmoid(model(xb)).cpu().numpy())
val_pred = np.concatenate(val_probs).ravel()

np.save("dcn_val_pred.npy", val_pred)
np.save("dcn_val_true.npy", y_va)

print(f"[Saved] Validation preds → dcn_val_pred.npy (len={len(val_pred)})")

# --- ✅ Create final submission file ---
id_col = "id" if "id" in test.columns else "row_id"

submit = pd.DataFrame({
    "ID": test[id_col],       # ← 여기서 무조건 'ID'로 강제 변경
    "clicked": test_pred
})
submit.to_csv("toss_dcn_v8_submit.csv", index=False)

print("[Saved] Final submission → toss_dcn_v8_submit.csv")



[Final] val_AP=0.065267 | val_WLL=0.636680
[Saved] Validation preds → dcn_val_pred.npy (len=2140834)
[Saved] Final submission → toss_dcn_v8_submit.csv
